In [1]:
from Selector import *
import sys
import time
from datetime import datetime
from view.HTML import show
import json
from indicators.MA import *
from indicators.TrendArea import *
from strategy.sequence.MAStrategyS import *
from strategy.sequence.RStrategyS import *
from simulation.SimulatorS import *
sys.path.append("../data")
import KDataDao as kds
from view.ViewDataFactory import *
from strategy.sequence.StrategyS1 import *
from strategy.component.buy.BuyA import *
from strategy.component.buy.BuyB import *
from strategy.component.buy.BuyC import *
from strategy.component.buy.BuyD import *
from strategy.component.buy.BuyBS import *
from strategy.component.buy.BuyZone import *
from strategy.component.sell.SellA import *
from strategy.component.sell.SellB import *
from strategy.component.sell.SellC import *
from strategy.component.sell.SellK import *
from strategy.component.sell.SellBS import *
from strategy.component.sell.SellZone  import *
from strategy.component.sell.SellS import *
from strategy.component.ConditionBean import *
from strategy.sequence.StrategyS import *

In [2]:
buys = [
#     BuyA(),
#             BuyB(),
#             BuyC(),
#             BuyD(),
#             BuyZone(),
            BuyBS()
]
sells = [
#             SellA(['a','b','c','d']),
#             SellB(['a','b','c','d']),
#             SellC(['a','b','c','d']),
#             SellK(['b','c','d']),
#             SellS(['a','b','c','d']),
#             SellZone(['z']),
            SellBS(['bs'])
]


In [3]:
class KStrategy:
    """
    K线策略
    """
    def __init__(self,trader=None,listK=None):
        """"""
        self.listK = listK
        self.trader = trader
        
    def init(self,trader,listK=[]):
        """"""
        self.listK = listK
        self.trader = trader
        
    def decide(self,k):
        if self.trader.isHold(k.code):
            if k.isDown():
                self.trader.sell(k.code, k.date, k.close, condition="k1")
        else:
            if k.isUp():
                self.trader.buy(k.code, k.date, k.close, condition="k1")

In [4]:
def firgueF(code):
    """显示股票"""
    clear_output()
    #获取数据
#     code = '002390'
#     code = '300051'
#     code = '002653'
#     code = '000782'
#     begin = "2013-06-05"
#     end = "2016-11-07"
    begin = "2014-11-02"
#     begin = "2015-06-30"
    end = '2015-07-24'
#     end = '2015-07-08'
    listK = kd.get_k_data(code,begin ,end)
    data,volume = ViewDataFactory.getCandlestick(listK)
    madic = ViewDataFactory.getMA(listK,(20,60))
    
    ts = StrategyS(buys,sells)
    sim = SimulatorS(ts)
    sim.runSingle(code,begin=begin,end=end)
    flags = ViewDataFactory.getTradeFlag(sim.trader.record,'#2AEBAF')
    earningsLine = sim.trader.earningsLine
    trendLine,trendMergeLine,trendBig = ViewDataFactory.trend2Line(ts.trend)
    areaRange = ViewDataFactory.trend2AreaRang(ts.trend,listK)
    
    mas = MAStrategyS(60)
    sim = SimulatorS(mas)
    sim.runSingle(code,begin=begin,end=end)
    maFlags = ViewDataFactory.getTradeFlag(sim.trader.record,'#2AEBAF')
    maEarningsLine = sim.trader.earningsLine
    
    ks = KStrategy()
    sim = SimulatorS(ks)
    sim.runSingle(code,begin=begin,end=end)
    kEarningsLine = sim.trader.earningsLine
    
    sim = SimulatorS(RStrategyS())
    sim.runSingle(code,begin=begin,end=end)
    rLine = sim.trader.earningsLine
    #获取开始显示的时间
    mDate = 0
    if len(data)>0:
        if len(listK)/2>abs(-250):
            mDate = data[-250][0]
        else:
            mDate = data[len(listK)/2][0]
            
    #显示
    params = {}
    params["code"] = code
    params["mDate"] = mDate
    params["data"] = json.dumps(data)
    params["volume"] = json.dumps(volume)
    params["earningsLine"] = json.dumps(earningsLine)
    params["maEarningsLine"] = json.dumps(maEarningsLine)
    params["kEarningsLine"] = json.dumps(kEarningsLine)
    params["rLine"] = json.dumps(rLine)
    params["maFlags"] = json.dumps(maFlags)
    params["flags"] = json.dumps(flags)
    params["line20"] = json.dumps(madic[20].line)
    params["line60"] = json.dumps(madic[60].line)
    params["trendLine"] = json.dumps(trendLine)
    params["trendMergeLine"] = json.dumps(trendMergeLine)
    params["trendBig"] = json.dumps(trendBig)
    params["areaRange"] = json.dumps(areaRange)
    
    show("StrategyArea.html",params)
    
#     show("StrategySingle.html",params)


In [5]:
class KB(Selector):
    """红绿红"""
    
    def analyze(self,list):
        """分析股票"""
        if True:
            return True
#         if (
#                         abs((list[-1].maxOC() - list[-3].maxOC()) / list[-3].maxOC()) < 0.03
#                 and list[-1].ratio() > 0.01
#             and list[-2].ratio() < -0.01
#         and list[-3].ratio() > 0.01):
#             return True
        mx,mn = HLRatio(len(list)-1,250,list)
        if mx > 1:
            return True
        return False
    
    def firgue(self,code):
        """显示股票"""
        firgueF(code)

In [6]:
kb = KB(False)
